Here i'm gonna compare the difference between this each image and the last one to detect if there is a pause in the video.

First i'm gonna select a test video that has an easy to spot pause

In [ ]:
import matplotlib.pyplot as plt
import cv2


import scipy

from pathlib import Path

import numpy as np

import h5py
import math


import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

#import datashader as ds


import holoviews as hv
from holoviews import opts
from holoviews.plotting.util import process_cmap
from bokeh.palettes import Viridis256
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')


import platform

import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../../..")


from pathlib import Path

import cv2

import json

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral11
from bokeh.io import output_notebook
import iqplot
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import LinearColorMapper, ColorBar

from Utilities.Utils import *
from Utilities.Processing import *
from Utilities.Ballpushing_utils import *
# Data loading
# Get the DataFolder

if platform.system() == "Darwin":
    DataPath = Path("/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")
# Linux Datapath
if platform.system() == "Linux":
    DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")

print(DataPath)
Folders = []
for folder in DataPath.iterdir():
    minfolder = str(folder).lower()
    if "tnt" in minfolder and "tracked" in minfolder and "pm" in minfolder:
        Folders.append(folder)

Folders
SavePath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/RasterPlots/")

from Utilities.Ballpushing_utils import *

Dataset_list = []
Flycount = 0

for folder in Folders:
    #print(f"Processing {folder}...")
    # Read the metadata.json file
    with open(folder / "Metadata.json", "r") as f:
        metadata = json.load(f)
        variables = metadata["Variable"]
        metadata_dict = {}
        for var in variables:
            metadata_dict[var] = {}
            for arena in range(1, 10):
                arena_key = f"Arena{arena}"
                var_index = variables.index(var)
                metadata_dict[var][arena_key] = metadata[arena_key][var_index]

        # In the metadata_dict, make all they Arena subkeys lower case

        for var in variables:
            metadata_dict[var] = {k.lower(): v for k, v in metadata_dict[var].items()}
        #print(metadata_dict)

        files = list(folder.glob("**/*.mp4"))

    for file in files:
        #print(file.name)
        # Get the arena and corridor numbers from the parent (corridor) and grandparent (arena) folder names
        arena = file.parent.parent.name
        # print(arena)
        corridor = file.parent.name

        # Get the Genotype and Dates from the metadata, arena should have a upper case first letter

        Genotype = metadata_dict["Genotype"][arena]
        #print(f"Genotype: {Genotype} for arena {arena}")

        Date = metadata_dict["Date"][arena]
        # print(f"Date: {Date} for arena {arena}")

        Light = metadata_dict["Light"][arena]
        FeedingState = metadata_dict["FeedingState"][arena]
        Period = metadata_dict["Period"][arena]

        start, end = np.load(file.parent / 'coordinates.npy')
        
        dir = file.parent

        # Define flypath as the *flytrack*.analysis.h5 file in the same folder as the video
        try:
            flypath = list(dir.glob("*flytrack*.analysis.h5"))[0]
            #print(flypath.name)
        except IndexError:
            #print(f"No fly tracking file found for {file.name}, skipping...")
            
            continue

        # Define ballpath as the *tracked*.analysis.h5 file in the same folder as the video
        try:
            ballpath = list(dir.glob("*tracked*.analysis.h5"))[0]
            #print(ballpath.name)
        except IndexError:
            #print(f"No ball tracking file found for {file.name}, skipping...")
            
            continue

        vidpath = file
        vidname = f"{Genotype}_{Date}_Light_{Light}_{FeedingState}_{Period}_{arena}_{corridor}"

        try:
            # Extract interaction events and mark them in the DataFrame
            data = extract_interaction_events(ballpath, flypath, mark_in_df=True)
            data["start"] = start
            data["end"] = end
            data["Genotype"] = Genotype
            data["Date"] = Date
            data["arena"] = arena
            data["corridor"] = corridor
            Flycount += 1
            data["Fly"] = f'Fly {Flycount}'
            # Compute yball_relative relative to start
            data['yball_relative'] = abs(data['yball_smooth'] - data['start'])

            # Fill missing values using linear interpolation
            data['yball_relative'] = data['yball_relative'].interpolate(method='linear')
            
            
            # Append the data to the all_data DataFrame
            Dataset_list.append(data)
        except Exception as e:
            error_message = str(e)
            traceback_message = traceback.format_exc()
            #print(f"Error processing video {vidname}: {error_message}")
            #print(traceback_message)

# Concatenate all dataframes in the list into a single dataframe
Dataset = pd.concat(Dataset_list, ignore_index=True)

Dataset.head()

In [ ]:
# Get a particular fly video from the dataset

TestFly = DataPath/'230803_TNTscreen_PM_1_Videos_Tracked/arena2/corridor3/corridor3.mp4'

DataFly = Dataset[(Dataset['Date'] == '230803') & (Dataset['arena'] == 'arena2') & (Dataset['corridor'] == 'corridor3')]

In [ ]:
import cv2
import numpy as np

# Open the video file
cap = cv2.VideoCapture(TestFly.as_posix())

# Initialize a variable to store the previous frame
prev_frame = None

# Initialize a list to store the sum of pixel differences for each frame
pixel_diffs = []

while True:
    # Read the next frame from the video
    ret, frame = cap.read()

    # If the frame was read successfully
    if ret == True:
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # If this is not the first frame
        if prev_frame is not None:
            # Compute the absolute difference between the current and previous frame
            diff = cv2.absdiff(gray, prev_frame)

            # Sum up all pixel differences and append to the list
            pixel_diffs.append(np.sum(diff))

        # Save the current frame for use in the next iteration
        prev_frame = gray

    else:
        # If there was an error reading the frame, we break the loop
        break

# Release the video file
cap.release()



In [ ]:
# plot the pixel differences

curve_diff = hv.Curve(pixel_diffs, 'Frame', 'Sum of pixel differences')

curve_fly = hv.Curve(DataFly, 'Frame', 'yfly_smooth')

curve_diff.opts(width=600, height=400, tools=['hover'], show_grid=True)
curve_fly.opts(width=600, height=400, tools=['hover'], show_grid=True)

curve_diff + curve_fly

In [ ]:
147*30

In [ ]:
# Add a column to the dataframe to store where the fly is pausing

# Calculate the difference between consecutive frames
Dataset['yfly_diff'] = Dataset['yfly_smooth'].diff().abs()

# Identify periods where the difference is less than 5 for at least 200 frames
Dataset['Pausing'] = (Dataset['yfly_diff'] < 0.05).rolling(200).sum() == 200

# Replace NaN values with False
Dataset['Pausing'].fillna(False, inplace=True)

In [ ]:
Dataset.head()

In [ ]:
DataFly = Dataset[(Dataset['Date'] == '230803') & (Dataset['arena'] == 'arena2') & (Dataset['corridor'] == 'corridor3')]

In [ ]:
# Create a new column 'PauseGroup' where change in 'Pausing' is detected
DataFly['PauseGroup'] = (DataFly['Pausing'] != DataFly['Pausing'].shift()).cumsum()

# Filter rows where 'Pausing' is True
pauses = DataFly[DataFly['Pausing'] == True]

# Group by 'PauseGroup' and get the first and last frame of each pause event
pause_events = pauses.groupby('PauseGroup')['time'].agg(['first', 'last'])

# Reset the index of the DataFrame
pause_events.reset_index(drop=True, inplace=True)

# Print the pause events
print(pause_events)

# Convert them to 


In [ ]:
# Convert 'first' and 'last' columns to datetime format
pause_events['first'] = pd.to_datetime(pause_events['first'], unit='s').dt.time
pause_events['last'] = pd.to_datetime(pause_events['last'], unit='s').dt.time

# Print the pause events
print(pause_events)

In [ ]:
pauses

In [ ]:
pause_events

In [ ]:
yflydiff = hv.Curve(DataFly, 'Frame', 'yfly_diff')

yflydiff.opts(width=600, height=400, tools=['hover'], show_grid=True)

In [ ]:
8700/30

In [ ]:
10800/30

In [ ]:
 200/30